Читаем файл и смотрим все ли правльно прочиталось

In [2]:
import pandas as pd


df = pd.read_excel('OTP_session_id.xlsx')
print(df.head())

    client_id            session_id   datetime    category
0  2397727872  b7fcc1f6-686b-41c005 2020-01-06   otp-login
1  2397727873  b7fcc1f6-686b-41c006 2020-01-06  mobile-otp
2  2397727874  b7fcc1f6-686b-41c007 2020-01-06     default
3  2397727875  b7fcc1f6-686b-41c008 2020-01-06  pass-login
4  2397727876  b7fcc1f6-686b-41c009 2020-01-06   otp-login


# Задание 1. 

Показать категорию, по которой было введено наибольшее число кодов за все время

In [3]:
df_1 = df.groupby('category')['session_id'].count()
print(df_1.idxmax())


mobile-otp


# Задание 2. 

Добавить индикатор, который будет выделять следующие значения:
•  Если otp для категории мобильная идентификация (mobile), то = 1
•  Если otp для категории логин (login), но не для логина с помощью пароля (pass), то = 2
Все остальные заявки не должны попасть в результат выполнения запроса.


In [4]:
needs = [df['category'] == 'mobile-otp',df['category'] == 'otp-login']
indikator = [1, 2]
df_2=df.copy()

df_2.loc[df_2['category'] == 'mobile-otp', 'indikator'] = 1
df_2.loc[df_2['category'] == 'otp-login', 'indikator'] = 2
df_2 = df_2[df_2['indikator'].notna()]

print(df_2.head())



    client_id            session_id   datetime    category  indikator
0  2397727872  b7fcc1f6-686b-41c005 2020-01-06   otp-login        2.0
1  2397727873  b7fcc1f6-686b-41c006 2020-01-06  mobile-otp        1.0
4  2397727876  b7fcc1f6-686b-41c009 2020-01-06   otp-login        2.0
5  2397727877  b7fcc1f6-686b-41c010 2020-01-06  mobile-otp        1.0
8  2397727880  b7fcc1f6-686b-41c013 2020-01-06   otp-login        2.0


# Задание 3. 

Посчитать метрику Month-of-Month (прирост текущего месяца к предыдущему) по уникальным клиентам с кодами otp-login.

In [5]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%m/%d/%Y')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month

df_3 = df[df['category'] == 'otp-login']
df_3 = df_3.groupby(['year', 'month'])['client_id'].nunique().reset_index(name='count')
df_3 = df_3.sort_values(['year', 'month'])
df_3['month_of_month'] =  df_3['count'] - df_3['count'].shift(1)


print(df_3)


   year  month  count  month_of_month
0  2020      1     86             NaN
1  2020      2    101            15.0
2  2020      3     99            -2.0


# Задание 4.  

сформируйте:
•  Количество введённых ОТП кодов в разрезе категории кода ОТП
•  Долю каждой категории по убыванию
•  Количество введённых ОТП с накопительным итогом
•  Общее количество введённых ОТП кодов

In [13]:
df_4 = df.groupby('category').size().reset_index(name='category_count')
full_count = df_4['category_count'].sum()
procent = (df_4['category_count'] * 100.0 / full_count).round(0).astype(int)
count = df_4['category_count'].cumsum()

df_4['procent'] = procent
df_4['count'] = count
df_4['full_count'] = full_count
df_4 = df_4.sort_values(by='category', ascending=False)

print(df_4)


     category  category_count  procent  count  full_count
3  pass-login             383       28   1349        1349
2   otp-login             286       21    966        1349
1  mobile-otp             390       29    680        1349
0     default             290       21    290        1349
